# ChromaDB learning 
## Purpose of this notebook
- Familiarize interfaces and functionalities of chromadb
## Steps 
1. Setup ChromaDB locally and establish connection with the server through code  : Done 
2. Explore key concepts of ChromaDB 
    - Concept of 'collections'

In [2]:
import chromadb
chroma_client = chromadb.Client()

# switch `create_collection` to `get_or_create_collection` to avoid creating a new collection every time
collection = chroma_client.get_or_create_collection(name="my_collection")
'''

'''
# switch `add` to `upsert` to avoid adding the same documents every time
collection.upsert(
    documents=[
        "red pineapple",
        "sweet oranges",
        "Francis Suarez decided to invest in x.com",
        "Donald Trump is the new president"
    ],
    ids=["id1", "id2","id3","id4"]
)

results = collection.query(
    query_texts=["This is a query document about florida"], # Chroma will embed this for you
    n_results=2 # how many results to return
)

print(results)

{'ids': [['id2', 'id4']], 'embeddings': None, 'documents': [['sweet oranges', 'Donald Trump is the new president']], 'uris': None, 'data': None, 'metadatas': [[None, None]], 'distances': [[1.7286731004714966, 1.8054571151733398]], 'included': [<IncludeEnum.distances: 'distances'>, <IncludeEnum.documents: 'documents'>, <IncludeEnum.metadatas: 'metadatas'>]}


## Collections 
- Similar to Table concept of relational database 
- Documents are stored in a collection
### What are documents 
- Smallest unit of information which can be embedded into a vector database
- Analogus to a row in RDB
- Choice of splitting information into document involves various constraints and considerations
    1. Context length of the embedding model and the pipeline which follows the retrieval :
        - Long content size of document may lead to bad performance for small context models. 
        - Trade of is between size of document and the overall information context. Small document split results in missing the overall picture of the information 


## Design of vector db collections 
We need seperate collections for Local search vector retrieval pipeline. Following are the potential collections 
1. Entity collection
    - This contains all the entities present in the graph
    - Entities should be grouped based on the generated communities
        - That is each Document may contain multiple entities. Hence combining random entities to generate document based on context limit doesn't make sense. 
2. Relationship vector store 
    - Same rationale for entities applies here also
    - Group relationships based on communities.
    - Split relationships in the communities into documents of fixed token count 
3. Text unit/chunk Collection
    - Directly store each text chunk into this store
    - Each chunk/unit becomes a document
4. Community retport vector store
    - Each community report is a document 

In [ ]:
import yaml
import networkx as nx
import os
import itertools
import chromadb

def print_first_n_items_simple(my_dict, n):
    for key, value in itertools.islice(my_dict.items(), n):
        print(f"{key}: {value}")


current_dir = os.getcwd()
graph = None
data:dict = None
# /home/vivekv/Documents/RD/TheProphet/Odysseus/retrieval_stratergies/experiments/tmw_dnp3_scl_um_graph.yml
with open(f"{current_dir}/tmw_dnp3_scl_um_graph.yml","r") as f:
    data = yaml.safe_load(f)
    graph = nx.node_link_graph(data)

if(graph):
    print("successfully loaded graph")
else:
    print("failed to load")

edges = data['links']
nodes = data['nodes']
print(nodes[:2])


# Collections initialization 
chroma_client = chromadb.Client()
collection_text_units = chroma_client.get_or_create_collection(name="GraphRAG_Text_Units") # Raw text units
collection_entities = chroma_client.get_or_create_collection(name="GraphRAG_Entities") # All entities will be stored here
collection_relationships = chroma_client.get_or_create_collection(name="GraphRAG_Relationships") # Relationship storage 
collection_community_reports = chroma_client.get_or_create_collection(name="GraphRAG_Community_Reports") # Community reports 


def entity_collections(collection_entities,entities_list:list):
    '''
    given a list of entities and collection, attach them to one or more documents and save to collection
    '''
    if not entities_list:
        raise ValueError(f"Missing entities list input")
    
    pass



successfully loaded graph
[{'description': 'A library that provides pre-written code to simplify the development of applications, particularly in the context of communication protocols. ; A library containing source code that provides support for various communication protocols, including IEC 60870-5. ; A library that provides functions for target applications to interact with, including general calls and callback functions. ; The Source Code Library is a component that interfaces with the Target Application (TA) through well-documented modules or header files. It provides a three-sided interface, with two sides representing calls back into the TA from the SCL Interface, and the third side representing calls into the SCL from the TA. ; A software framework used for developing applications that interact with various devices and systems. ; A software library that provides functionalities for handling database operations, message processing, and other tasks. It supports both synchronous a

/home/vivekv/anaconda3/envs/research_asst/lib/python3.12/site-packages/networkx/readwrite/json_graph/node_link.py:287: FutureWarning: 
The default value will be changed to `edges="edges" in NetworkX 3.6.

To make this warning go away, explicitly set the edges kwarg, e.g.:

  nx.node_link_graph(data, edges="links") to preserve current behavior, or
  nx.node_link_graph(data, edges="edges") for forward compatibility.
  warnings.warn(


In [ ]:
#---------------Token counter-Begin-------------#
def _estimate_tokens_hf(text: str, model_name: str = "Qwen/Qwen2.5-Coder-14B") -> int:
    """
    Estimate the number of tokens using Hugging Face tokenizers.

    Args:
        text (str): The input text.
        model_name (str): The Hugging Face model name or path.
    
    Returns:
        int: The estimated number of tokens.
    """
    from transformers import AutoTokenizer
    try:
        tokenizer = AutoTokenizer.from_pretrained(model_name)
        tokens = tokenizer(text, return_tensors="pt")["input_ids"]
        return tokens.size(1)  # Token count
    except Exception as e:
        raise RuntimeError(f"Error loading Hugging Face tokenizer for {model_name}: {e}")
    
def _estimate_tokens_spm(text: str, model_path: str) -> int:
    """
    Estimate tokens using SentencePiece tokenizer.

    Args:
        text (str): The input text.
        model_path (str): Path to the SentencePiece model file (.model).

    Returns:
        int: Estimated token count.
    """
    import sentencepiece as spm
    try:
        sp = spm.SentencePieceProcessor(model_file=model_path)
        tokens = sp.encode(text, out_type=str)
        return len(tokens)
    except Exception as e:
        raise RuntimeError(f"Error loading SentencePiece model from {model_path}: {e}")
    
def _estimate_tokens_tiktoken(text: str, model: str = "gpt-4",fallback: str = "cl100k_base") -> int:
    """
    Estimate the number of tokens in a text string using the tiktoken library.

    Args:
        text (str): The input text to estimate token count for.
        model (str): The model name to determine the tokenizer (default: "gpt-4").
    
    Returns:
        int: The estimated number of tokens.
    """
    import tiktoken
    try:
        tokenizer = tiktoken.encoding_for_model(model)
    except KeyError:
        # Use fallback model
        tokenizer = tiktoken.get_encoding(fallback)
    return len(tokenizer.encode(text))
    
    # Encode the text and count the tokens
    token_count = len(tokenizer.encode(text))
    return token_count

def estimate_tokens(text: str, method: str = "hf", **kwargs) -> int:
    """
    Estimates the number of tokens in a given text using a specified tokenization method.

    This function acts as a wrapper for multiple token estimation methods and dynamically selects the appropriate method based on the `method` parameter. It logs the token estimation process, including the chosen method and the token count. If the token count is zero, a warning is logged.

    Args:
        text (str): 
            The input text for which the token count is to be estimated.
        method (str, optional): 
            The tokenization method to use. Defaults to `"hf"`. Supported methods are:
            - `"hf"`: Hugging Face tokenization.
            - `"spm"`: SentencePiece tokenization.
            - `"tiktoken"`: OpenAI's TikToken tokenization.
        **kwargs: 
            Additional keyword arguments passed to the underlying tokenization method for customization.

    Returns:
        int: 
            The estimated number of tokens in the input text.

    Raises:
        ValueError: 
            If an unsupported tokenization method is specified.

    Workflow:
        1. Log the selected tokenization method.
        2. Use the specified method to estimate the number of tokens:
            - If `method` is `"hf"`, call `_estimate_tokens_hf`.
            - If `method` is `"spm"`, call `estimate_tokens_spm`.
            - If `method` is `"tiktoken"`, call `estimate_tokens_tiktoken`.
        3. Log a warning if the estimated token count is zero.
        4. Log the final token count and return it.

    Sub-functions:
        - `_estimate_tokens_hf(text: str, **kwargs) -> int`: Estimates tokens using Hugging Face tokenization.
        - `estimate_tokens_spm(text: str, **kwargs) -> int`: Estimates tokens using SentencePiece tokenization.
        - `estimate_tokens_tiktoken(text: str, **kwargs) -> int`: Estimates tokens using OpenAI's TikToken.

    Example Usage:
        ```python
        text = "This is a sample text for token estimation."
        
        # Using Hugging Face tokenization
        token_count_hf = estimate_tokens(text, method="hf")
        
        # Using SentencePiece tokenization
        token_count_spm = estimate_tokens(text, method="spm")
        
        # Using TikToken tokenization
        token_count_tiktoken = estimate_tokens(text, method="tiktoken")
        ```

    Example Output:
        For the input text `"This is a sample text for token estimation."`, the token count may vary based on the method:
        - Hugging Face: 10 tokens
        - SentencePiece: 9 tokens
        - TikToken: 11 tokens

    Logging:
        - Logs the selected tokenization method at the beginning.
        - Logs a warning if the token count is zero, along with the input text.
        - Logs the estimated token count.

    Notes:
        - The function assumes that the tokenization methods `_estimate_tokens_hf`, `estimate_tokens_spm`, and `estimate_tokens_tiktoken` are implemented and available in the codebase.
        - The choice of tokenization method should align with the downstream processing system.

    Limitations:
        - The token count may differ across methods due to variations in tokenization algorithms.
        - If the input text is empty or poorly formatted, the token count may be zero, triggering a warning.
    """
    # logging.info(f"Estimating tokens using {method} method.")
    if method == "hf":
        token_count = _estimate_tokens_hf(text, **kwargs)
    elif method == "spm":
        token_count = _estimate_tokens_spm(text, **kwargs)
    elif method == "tiktoken":
        token_count = _estimate_tokens_tiktoken(text, **kwargs)
    else:
        raise ValueError(f"Unsupported tokenization method: {method}")
    # if(token_count==0):
    #     logging.warning(f"Zero token count for following chunk:\n{text}\n")
    # logging.info(f"Token count: {token_count}")
    return token_count
#---------------Token counter-End---------------#




## **🔹 Core Interfaces for ChromaDB Wrapper**
| **Functionality** | **Purpose** |
|------------------|------------|
| **1️⃣ Store Document** | Add a document (text + metadata + embedding) to ChromaDB. |
| **2️⃣ Retrieve Document by ID** | Fetch a document using its unique ID. |
| **3️⃣ Query Documents by Metadata** | Filter and retrieve documents based on metadata fields. |
| **4️⃣ Semantic Search (Vector Search)** | Retrieve similar documents using embeddings. |
| **5️⃣ Update Document** | Modify an existing document’s metadata or embedding. |
| **6️⃣ Delete Document** | Remove a document from ChromaDB. |
| **7️⃣ Collection Management** | Create, list, and delete collections in ChromaDB. |


# Document store



In [1]:
import chromadb
import uuid
import numpy as np
from typing import Optional, List

def store_document(collection, text: str, metadata: dict, embedding: Optional[List[float]] = None) -> str:
    """
    Stores a document in ChromaDB.

    Args:
        collection: The ChromaDB collection.
        text (str): The document content.
        metadata (dict): Metadata associated with the document.
        embedding (Optional[List[float]]): Precomputed embedding (optional).

    Returns:
        str: Unique document ID.
    """
    # Generate a unique document ID
    doc_id = metadata.get("id", str(uuid.uuid4()))
    
    # Compute embedding if not provided
    if embedding is None:
        embedding = np.random.rand(1536).tolist()  # Placeholder: Replace with real embedding model
    
    # Add text to metadata for retrieval context
    metadata["text"] = text

    # Store document in ChromaDB
    collection.add(embeddings=[embedding], metadatas=[metadata], ids=[doc_id])
    
    return doc_id


# Test code 
client = chromadb.PersistentClient(path="./chroma_db")
collection = client.get_or_create_collection("knowledge_base")

text = "Artificial intelligence is transforming the world."
metadata = {"source": "article", "title": "AI Revolution"}

doc_id = store_document(collection, text, metadata)
print(f"Stored document with ID: {doc_id}")

Stored document with ID: f13e284f-ea24-4c72-b995-107d24c67ac0


Detailed design of localsearch vector store implementation is done in corresponding files. Scope of this notebook is experiments with chromadb. 

# Filtering results with where clause ([Source](https://cookbook.chromadb.dev/core/filters/#in))

In [ ]:
import chromadb
from chromadb.utils import embedding_functions
from chromadb.config import DEFAULT_TENANT, DEFAULT_DATABASE, Settings
from typing import List,Dict, Any
import torch
import yaml

class LocalSearchUtils():
    def __init__(self,source,client):
        self.client = client
        self.source = source
        self.embedding_function = embedding_functions.SentenceTransformerEmbeddingFunction(
                                                    model_name="all-MiniLM-L6-v2")
        self.collections_dict = self.setup_vector_databases(client=self.client)
        self.collections_dict = self.load_collections(source=self.source,collections_dict=self.collections_dict)

    def setup_vector_databases(self,client):
        text_units_collection = client.get_or_create_collection(name="text_units",embedding_function=self.embedding_function)
        entities_collection = client.get_or_create_collection(name="entities",embedding_function=self.embedding_function)
        entity_descriptions = client.get_or_create_collection(name="entity_descriptions",embedding_function=self.embedding_function)
        relationships_collection = client.get_or_create_collection(name="relationships",embedding_function=self.embedding_function)
        community_summaries_collection = client.get_or_create_collection(name="community_summaries",embedding_function=self.embedding_function)
        return {"text_units":text_units_collection,"entities":entities_collection,"entity_descriptions":entity_descriptions,
                "relationships":relationships_collection,"community_summaries":community_summaries_collection}

    def load_collections(self,source,collections_dict):
        cwd = os.getcwd()
        artifacts_path = os.path.join(cwd,"../../../artifacts/")
        community_path = f"{artifacts_path}graphRAG/community/{source}/{source}_communities.pt"
        graph_path = f"{artifacts_path}graph_extraction/{source}/{source}_graph.yml"
        text_units_path = f"{artifacts_path}graph_extraction/{source}/{source}_chunks.yml"

        all_communities = torch.load(community_path)
        with(open(graph_path,"r")) as loc_file:
            graph = yaml.safe_load(loc_file)

        with(open(text_units_path,"r")) as loc_file:
            text_units = yaml.safe_load(loc_file)
        # entities and communities can be collected from all_communities 
        # text units should be collected from text_chunks file
        # relationships should be collected from graph.yml file

        entities = {}
        for key,community in all_communities.items():
            if(key in ['needs_node_summary','needs_community_summary']):
                continue
            # entities|=community['nodes']
            for entity_name, entity_data in community['nodes'].items():
                entity_data['community'] = key  # Embed the community name inside the entity data
                entities[entity_name] = entity_data
        relationships = graph['links']
        
        # Logic to check if the collections contain any values or not. 

        collections_dict['community_summaries'] = self.store_communities(collection=collections_dict['community_summaries'],communities=all_communities)
        collections_dict['entities']= self.store_entities(collection=collections_dict['entities'],entities=entities)
        collections_dict['entity_descriptions'] = self.store_entity_descriptions(collection=collections_dict['entity_descriptions'],entities=entities)
        collections_dict['relationships'] = self.store_relationships(collection=collections_dict['relationships'],relationships=relationships)
        collections_dict['text_units'] = self.store_text_units(collection=collections_dict['text_units'],text_unit_list=text_units)
        return collections_dict

    #---------------store-Begin-------------#
    def store_text_units(self,collection, text_unit_list: List[str]):
        """
        Stores text units as ChromaDB documents in the given collection.

        Args:
            collection: ChromaDB collection where text units will be stored.
            text_unit_list (List[str]): List of extracted text units.

        Returns:
            The updated ChromaDB collection.
        """
        # Handle empty input
        if not text_unit_list:
            return collection  # No change
        existing_docs = collection.get(include=["documents"])
        existing_ids = existing_docs["ids"] if existing_docs and "ids" in existing_docs else []
        # Generate document IDs using the text unit index
        doc_ids = [str(i) for i in range(len(text_unit_list))]
        missing_ids = [item for item in doc_ids if item not in existing_ids]
        print(f"Missing text_unit ids:{missing_ids}")
        if(missing_ids):
            # Store text units in ChromaDB
            collection.upsert(
                ids=doc_ids,
                documents=text_unit_list
            )
        return collection

    def store_entity_descriptions(self,collection, entities: Dict[str, Dict[str, str]]):
        """
        Stores entity nodes into a ChromaDB collection.

        Args:
            collection: The ChromaDB collection where entities will be stored.
            entities (Dict[str, Dict[str, str]]): A dictionary where:
                - Each key is a node name (acting as the node ID).
                - Each value is a dictionary containing:
                    - "description" (str): A detailed description of the entity.
                    - "summary" (str): A concise summary of the description.
                    - "type" (str): The category/type of the entity.

        Processing:
        - Each entity is converted into a document format for storage.
        - The node name is used as the unique document ID.
        - The "type" field is attached as metadata.
        - The stored document consists of the concatenated "summary" and "description".

        Returns:
            The updated ChromaDB collection.
        """

        # Handle empty input
        if not entities:
            return collection  # No change
        existing_docs = collection.get(include=["documents"])
        existing_ids = existing_docs["ids"] if existing_docs and "ids" in existing_docs else []

        # Prepare data for batch insertion
        doc_ids = list(entities.keys())  # Node names as IDs
        missing_doc_ids = [item for item in doc_ids if item not in existing_ids]
        print(f"Following are the missing_doc_ids:{missing_doc_ids}")
        documents = [f"{entities[id]['summary']}\n{entities[id]['description']}" for id in missing_doc_ids]
        metadata_list = [{"type": entities[id]["type"]} for id in missing_doc_ids]

        if(missing_doc_ids):
            # Store entities in ChromaDB
            collection.upsert(
                ids=doc_ids,
                documents=documents,
                metadatas=metadata_list
            )
        return collection

    def store_entities(self,collection,entities):
        # Handle empty input
        if not entities:
            return collection  # No change
        existing_docs = collection.get(include=["documents"])
        existing_ids = existing_docs["ids"] if existing_docs and "ids" in existing_docs else []

        # Prepare data for batch insertion
        doc_ids = list(entities.keys())  # Node names as IDs
        missing_doc_ids = [item for item in doc_ids if item not in existing_ids]

        print(f"Following are the missing_doc_ids:{missing_doc_ids}")

        documents = [f"{entity}" for entity in entities.keys()]
        metadata_list = [{"type": entities[id]["type"]} for id in missing_doc_ids]

        if(missing_doc_ids):
            # Store entities in ChromaDB
            collection.upsert(
                ids=doc_ids,
                documents=documents,
                metadatas=metadata_list
            )
        return collection

    def store_relationships(self,collection, relationships: List[Dict[str, str]]):
        """
        Stores relationship data into a ChromaDB collection.

        Args:
            collection: The ChromaDB collection where relationships will be stored.
            relationships (List[Dict[str, str]]): A list of relationship dictionaries, 
                where each dictionary contains:
                - "description" (str): A textual explanation of the relationship.
                - "source" (str): The starting node of the relationship.
                - "target" (str): The ending node of the relationship.
                - "weight" (float): A numerical value representing the relationship strength.

        Processing:
        - First, check which relationship IDs are **already stored** in the collection.
        - Filter out existing IDs and insert **only missing relationships**.
        - The unique ID for each relationship is generated in the format: "(<source>,<target>)".
        - The "description" field is stored as the document content.
        - The metadata includes:
            - "source" (str): The starting node.
            - "target" (str): The ending node.
            - "weight" (float): Strength of the relationship.

        Returns:
            The updated ChromaDB collection.
        """

        # Handle empty input
        if not relationships:
            return collection  # No change

        # Step 1: Get existing document IDs in the collection
        existing_docs = collection.get(include=["documents"])
        existing_ids = set(existing_docs["ids"]) if existing_docs and "ids" in existing_docs else set()

        doc_ids = []
        documents = []
        metadata_list = []

        # Step 2: Filter out already existing relationships
        for rel in relationships:
            if "description" in rel and "source" in rel and "target" in rel and "weight" in rel:
                # Generate unique relationship ID
                rel_id = f"({rel['source']},{rel['target']})"

                if rel_id in existing_ids:
                    continue  # Skip existing relationships

                # Append only missing relationships
                doc_ids.append(rel_id)
                documents.append(rel["description"])
                metadata_list.append({
                    "source": rel["source"],
                    "target": rel["target"],
                    "weight": rel["weight"]
                })

        # Step 3: Store only new relationships in ChromaDB
        if doc_ids:  # Only store if there are new relationships
            collection.upsert(
                ids=doc_ids,
                documents=documents,
                metadatas=metadata_list
            )

        return collection

    def store_communities(self,collection, communities: Dict[str, Dict[str, str]]):
        """
        Stores community data into a ChromaDB collection.

        Args:
            collection: The ChromaDB collection where communities will be stored.
            communities (Dict[str, Dict[str, str]]): A dictionary containing various keys,
                but only the following keys are of interest:
                - "title" (str): The name or title of the community (used as metadata).
                - "summary" (str): A brief description summarizing the community.
                - "metadata" (dict): A dictionary containing:
                    - "community_id" (int): A unique ID derived from the GraphRAG community.
                    - "node_count" (int): Number of nodes in the community.
                    - "internal_edge_count" (int): Number of internal edges.
                    - "external_edge_count" (int): Number of external edges.
                    - "list_of_nodes" (list(str)): List of member nodes
        Processing:
        - First, check which community IDs are **already stored** in the collection.
        - Filter out existing IDs and insert **only missing communities**.
        - The "community_id" is used as the **unique document ID**.
        - The "summary" is stored as the document content.
        - Metadata includes:
            - "title" (str): The title of the community.
            - "node_count" (int): Total number of nodes.
            - "total_edges" (int): Sum of "internal_edge_count" and "external_edge_count".

        Returns:
            The updated ChromaDB collection.
        """

        # Handle empty input
        if not communities:
            return collection  # No change

        # Step 1: Get existing document IDs in the collection
        existing_docs = collection.get(include=["documents"])
        existing_ids = existing_docs["ids"] if existing_docs and "ids" in existing_docs else []
        missing_ids = communities.keys() - existing_ids
        print(f"Missing communities in the collection: {missing_ids}")

        doc_ids = []
        documents = []
        metadata_list = []

        # Step 2: Filter out already existing communities
        for community_id in missing_ids:
            data = communities[community_id]
            if "title" in data and "summary" in data and "metadata" in data: # avoid other keys except communities from the dictionary 
                # Convert community_id to string to match stored document IDs
                community_id_str = str(community_id)
                title = data["title"]
                summary = data["summary"]
                metadata = data["metadata"]
                metadata["nodes"] = [node for node in data["nodes"].keys()] 

                # Extract node count and total edges
                node_count = metadata.get("node_count", 0)
                total_edges = metadata.get("internal_edge_count", 0) + metadata.get("external_edge_count", 0)

                # Append only missing communities
                doc_ids.append(community_id_str)
                documents.append(summary)
                metadata_list.append({"node_count": node_count, "title": title, "total_edges": total_edges})

        # Step 3: Store only new communities in ChromaDB
        if doc_ids:  # Only store if there are new communities
            print(f"Adding following doc_ids to collection: {doc_ids}")
            collection.upsert(
                ids=doc_ids,
                documents=documents,
                metadatas=metadata_list
            )

        return collection
    #---------------store-End---------------#

    #---------------search-Begin-------------#
    def search_text_units(self,collection, query: str, k: int = 3) -> List[str]:
        """
        Given a query text, retrieve the top k most relevant text units from the collection.

        Args:
            collection: The ChromaDB collection for text units.
            query (str): The search query.
            k (int): The number of top results to retrieve.

        Returns:
            List[str]: A list of top k matching text units.
        """
        results = collection["text_units"].query(query_texts=[query], n_results=k)
        return results["documents"][0] if results and "documents" in results else []

    def search_entities(self, query: str,collection_dict: Dict[str, Any]= None, k: int = 3) -> Dict[str, Dict[str, Dict[str, Any]]]:
        """
        Searches for the most relevant entities based on a given query.

        This function performs vector similarity search on both:
        1. **Entity Names Collection** – Finds entities directly by name.
        2. **Entity Descriptions Collection** – Finds entities based on their descriptions.

        The results from both sources are merged using **harmonic mean** to prioritize entities that appear in both searches.

        Args:
            collection_dict (Dict[str, Any]): A dictionary of ChromaDB collections containing:
                - "entities": ChromaDB collection for entity names.
                - "entity_descriptions": ChromaDB collection for entity descriptions.
            query (str): The search query used for entity retrieval.
            k (int): The number of top results to retrieve from each search (default: 3).

        Returns:
            Dict[str, Dict[str, Dict[str, Any]]]: A dictionary containing:
                - "entity_name_store": Entities retrieved based on names.
                - "entity_desc_store": Entities retrieved based on descriptions.
                Each entity is stored as:
                ```json
                {
                    "<entity_name>": {
                        "type": "<entity_type>",
                        "distance": <retrieval_distance>,
                        "description": "<entity_description>"
                    }
                }
                ```

        Example:
            ```python
            collection_dict = {
                "entities": chromadb.PersistentClient().get_collection("entity_names"),
                "entity_descriptions": chromadb.PersistentClient().get_collection("entity_descriptions"),
            }
            results = search_entities(collection_dict, "AI research", k=3)
            print(results)
            ```
        """

        def reconstruct_entities(retrieved_docs: Dict[str, Any], distances: List[float] = []) -> Dict[str, Dict[str, Any]]:
            """
            Reconstructs the entities dictionary from a ChromaDB retrieval response.

            This function extracts entity names, types, descriptions, and retrieval distances
            to form a structured dictionary.

            Args:
                retrieved_docs (Dict[str, Any]): The dictionary returned from a ChromaDB retrieval query.
                distances (List[float], optional): A list of distances (lower means higher similarity).
                    - If provided, it overrides the distances extracted from `retrieved_docs`.

            Returns:
                Dict[str, Dict[str, Any]]: A dictionary where each key is an entity name, and the value contains:
                    - "type": The entity classification.
                    - "distance": The similarity score (lower is better).
                    - "description": A detailed explanation of the entity.

            Example Output:
                ```json
                {
                    "TOOLNODE": {
                        "type": "TECHNOLOGIES AND FRAMEWORKS",
                        "distance": 0.6034,
                        "description": "A prebuilt component in LangGraph that handles tools returning `Command` objects."
                    }
                }
                ```
            """

            entities = {}

            if distances:
                ids_list = retrieved_docs.get("ids", [[]])  # List of entity names
                documents_list = retrieved_docs.get("documents", [[]])  # List of descriptions
                metadatas_list = retrieved_docs.get("metadatas", [[]])  # List of metadata dictionaries
                distances_list = distances  # Use provided distances
            else:
                ids_list = retrieved_docs.get("ids", [[]])[0]
                documents_list = retrieved_docs.get("documents", [[]])[0]
                metadatas_list = retrieved_docs.get("metadatas", [[]])[0]
                distances_list = retrieved_docs.get("distances", [[]])[0]

            num_results = min(len(ids_list), len(documents_list), len(metadatas_list), len(distances_list))

            for i in range(num_results):
                entity_name = ids_list[i]
                entity_type = metadatas_list[i].get("type", "UNKNOWN") if metadatas_list else "UNKNOWN"
                distance = distances_list[i]
                description = documents_list[i]

                entities[entity_name] = {
                    "type": entity_type,
                    "distance": distance,
                    "description": description
                }

            return entities

        def merge_retrieved_dicts(dict1: Dict[str, Dict[str, Any]], dict2: Dict[str, Dict[str, Any]]):
            """
            Merges two retrieved entity dictionaries based on common keys.

            If an entity appears in both dictionaries, their 'distance' values are updated using **harmonic mean**:
                \[
                new\_distance = \frac{2}{(\frac{1}{d1} + \frac{1}{d2})}
                \]
            This ensures that frequently appearing entities are given **higher priority** in the retrieval results.

            Args:
                dict1 (Dict[str, Dict[str, Any]]): The primary dictionary to update.
                dict2 (Dict[str, Dict[str, Any]]): The secondary dictionary, from which matching elements are merged and removed.

            Returns:
                None: dict1 is updated in place, and matching keys are removed from dict2.

            Example:
                ```python
                dict1 = {"TOOLNODE": {"distance": 0.60}}
                dict2 = {"TOOLNODE": {"distance": 0.75}}
                merge_retrieved_dicts(dict1, dict2)
                print(dict1)  # TOOLNODE distance will be updated to ~0.667
                ```
            """
            common_keys = set(dict1.keys()) & set(dict2.keys())  # Find common entities

            for key in common_keys:
                d1 = dict1[key]["distance"]
                d2 = dict2[key]["distance"]

                # Compute harmonic mean of distances (lower is better)
                new_distance = 2 / ((1 / d1) + (1 / d2))

                dict1[key]["distance"] = new_distance  # Update dict1
                del dict2[key]  # Remove from dict2
        
        if(not collection_dict):
            collection_dict = self.collections_dict

        # Step 1: Query both collections (entity names and descriptions)
        entity_name_retrievals = collection_dict["entities"].query(query_texts=[query], n_results=k)
        entity_desc_retrievals = collection_dict["entity_descriptions"].query(query_texts=[query], n_results=k)

        # Step 2: Process retrievals
        results = {"entity_name_store": {}, "entity_desc_store": {}}

        if entity_name_retrievals:
            entities_to_get = entity_name_retrievals.get("ids", [[]])[0]
            distances = entity_name_retrievals.get("distances", [[]])[0]
            name_retrievals_with_desc = collection_dict["entity_descriptions"].get(ids=entities_to_get)
            results["entity_name_store"] = reconstruct_entities(name_retrievals_with_desc, distances=distances)

        if entity_desc_retrievals:
            results["entity_desc_store"] = reconstruct_entities(entity_desc_retrievals)

        # Step 3: Merge entity name and description retrievals
        merge_retrieved_dicts(results["entity_name_store"], results["entity_desc_store"])

        return results

    def search_relationships(self,collection,query, **kwargs) -> List[str]:
        """
        Given a query text, retrieve the top k most relevant relationships from the collection.

        Args:
            collection: The ChromaDB collection for relationships.
            query (str): The search query.
            k (int): The number of top results to retrieve.

        Returns:
            List[str]: A list of top k matching relationships.
        """
        if query:
            results = collection["relationships"].query(**kwargs)
        else:
            results = collection["relationships"].get(**kwargs)
        return results

    def search_community_summaries(self,collection, query: str, k: int = 3) -> List[str]:
        """
        Given a query text, retrieve the top k most relevant community summaries from the collection.

        Args:
            collection: The ChromaDB collection for community summaries.
            query (str): The search query.
            k (int): The number of top results to retrieve.

        Returns:
            List[str]: A list of top k matching community summaries.
        """
        results = collection["relationships"].query(query_texts=[query], n_results=k)
        return results["documents"][0] if results and "documents" in results else []


import os 
cwd = os.getcwd()
client = chromadb.PersistentClient(
                path=f"{cwd}../graphRAG_vector_db",
                settings=Settings(),
                tenant=DEFAULT_TENANT,
                database=DEFAULT_DATABASE,
            )

# source = "tmw_sdg_um"
source = "low_level"
loc_search_db = LocalSearchUtils(client=client,source=source)

entities = ["STATEGRAPH","MESSAGEGRAPH","LANGGRAPH","PERSISTENCE LAYER","TOOLNODE"] # get from state
#---------------Query to get relationships-Begin-------------#
relationships = loc_search_db.search_relationships(collection=loc_search_db.collections_dict,query=False,
    where={"$or":[{"source": {"$in": entities}},{"target": {"$in": entities}}]},
    include=["documents","metadatas"]
    )
#---------------Query to get relationships-End---------------#
#---------------Query to get all communities-Begin-------------#
# communities = loc_search_db.search_communities(collection=loc_search_db.collections_dict,query=False,
#     where={"nodes": {"$in": entities}},
#     include=["documents","metadatas"]
#     )
#---------------Query to get all communities-End---------------#

print(f"Following are the identified relationships:\n{relationships}")
print(f"lenth of retrievals:{len(relationships['ids'])}")
# for reln in relationships['ids'][0]:
#     print(f"{reln}\n")

<>:420: SyntaxWarning: invalid escape sequence '\['
<>:420: SyntaxWarning: invalid escape sequence '\['
/tmp/ipykernel_4038038/2147318356.py:420: SyntaxWarning: invalid escape sequence '\['
  """
/home/vivekv/anaconda3/envs/research_asst/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/tmp/ipykernel_4038038/2147318356.py:33: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits th

Missing communities in the collection: {'community_1', 'community_12', 'community_4', 'community_3', 'community_7', 'community_8', 'community_10', 'needs_node_summary', 'community_2', 'community_9', 'community_5', 'community_0', 'needs_community_summary', 'community_11', 'community_6'}
Adding following doc_ids to collection: ['community_1', 'community_12', 'community_4', 'community_3', 'community_7', 'community_8', 'community_10', 'community_2', 'community_9', 'community_5', 'community_0', 'community_11', 'community_6']
Following are the missing_doc_ids:['INPUT SCHEMA', 'START NODE', 'STATEGRAPH', 'OUTPUT SCHEMA', 'GRAPH', 'RUNNABLELAMBDA', 'END NODE', 'NODES', 'END', 'MAP-REDUCE DESIGN PATTERN', 'EDGES', 'COGNITIVE ARCHITECTURE', 'MESSAGE OBJECTS', 'ANYMESSAGE', 'STATE', 'MESSAGEGRAPH', 'SCHEMA', 'PYDANTIC MODEL', 'REDUCER FUNCTIONS', 'OPERATOR.ADD', 'ANNOTATED', 'CUSTOMER SUPPORT APPLICATION', 'DEFAULT REDUCER', 'GRAPH STATE', 'MESSAGESSTATE', 'TYPEDDICT', 'ADD_MESSAGES FUNCTION', 'M

## Findings
- If `n_results` is not specified, query return top 10 results by default.
- `n_results` parameter is not available for get method
- `get` method is used to retrieve blindly without vector search. `query` does vector search